In [1]:
# Copyright (c) MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Denoising Diffusion Probabilistic Models with MedNIST Dataset

This tutorial illustrates how to use MONAI for training a denoising diffusion probabilistic model (DDPM)[1] to create
synthetic 2D images.

[1] - Ho et al. "Denoising Diffusion Probabilistic Models" https://arxiv.org/abs/2006.11239


## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[tqdm]"
!python -c "import matplotlib" || pip install -q matplotlib
############################################################
!pip install -q medmnist
############################################################
%matplotlib inline

<frozen importlib._bootstrap_external>:1301: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
2025-12-17 20:02:30.546665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766001750.568781   18644 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766001750.575453   18644 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766001750.592783   18644 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766001750.592820   18644 computation_placer.cc:1

## Setup imports

In [ ]:
import os
import shutil
import tempfile
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from monai import transforms
from monai.config import print_config
from monai.data import Dataset, CacheDataset, DataLoader
from monai.utils import first, set_determinism
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm

############################################################
#from monai.apps import MedNISTDataset
from medmnist import INFO
from medmnist import PathMNIST
from torchvision import transforms as tv_transforms
############################################################

############################################################
#from generative.inferers import DiffusionInferer
#from generative.networks.nets import DiffusionModelUNet
#from generative.networks.schedulers import DDPMScheduler
from monai.inferers import DiffusionInferer
from monai.networks.nets import DiffusionModelUNet
from monai.networks.schedulers import DDPMScheduler
############################################################

############################################################
import cv2
############################################################

print_config()

## Visualizamos las clases de PathMNIST

In [ ]:
dataset_info = INFO['pathmnist']
label_dict = dataset_info['label']

preview_data = PathMNIST(split="train", download=True)

fig, axes = plt.subplots(1, 9, figsize=(20, 5))
found_classes = {}

for img, label in zip(preview_data.imgs, preview_data.labels):
    idx = label[0]
    if idx not in found_classes:
        found_classes[idx] = img
    if len(found_classes) == 9: break

for i in range(9):
    axes[i].imshow(found_classes[i])
    axes[i].set_title(f"{i}: {label_dict[str(i)]}", fontsize=10)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## Setup data directory

You can specify a directory with the MONAI_DATA_DIRECTORY environment variable.

This allows you to save results and reuse downloads.

If not specified a temporary directory will be used.

In [ ]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

## Set deterministic training for reproducibility

In [ ]:
set_determinism(42)

## Setup MedNIST Dataset and training and validation dataloaders
In this tutorial, we will train our models on the MedNIST dataset available on MONAI
(https://docs.monai.io/en/stable/apps.html#monai.apps.MedNISTDataset). In order to train faster, we will select just
one of the available classes ("Hand"), resulting in a training set with 7999 2D images.

In [ ]:
############################################################
#train_data = MedNISTDataset(root_dir=root_dir, section="training", download=True, progress=False, seed=0)
#train_datalist = [{"image": item["image"]} for item in train_data.data if item["class_name"] == "Hand"]

CLASE_ELEGIDA = 7

train_data = PathMNIST(split="train", download=True, size=28)
train_datalist = [
    {"image": img}
    for img, label in zip(train_data.imgs, train_data.labels)
    if label[0] == CLASE_ELEGIDA
][:8000]
############################################################

Here we use transforms to augment the training dataset:

1. `LoadImaged` loads the hands images from files.
1. `EnsureChannelFirstd` ensures the original data to construct "channel first" shape.
1. `ScaleIntensityRanged` extracts intensity range [0, 255] and scales to [0, 1].
1. `RandAffined` efficiently performs rotate, scale, shear, translate, etc. together based on PyTorch affine transform.

In [ ]:
############################################################
def rgb_to_gray_cv2(d):
    img = d["image"]
    if img.ndim == 3 and img.shape[2] == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # H,W
        d["image"] = gray[:, :, None] # agrega dimensión de canal -> 1,H,W
    return d
############################################################

train_transforms = transforms.Compose(
    [
        ############################################################
        #transforms.LoadImaged(keys=["image"]),
        #transforms.EnsureChannelFirstd(keys=["image"]),
        #transforms.Lambda(func=rgb_to_gray_cv2),
        transforms.EnsureChannelFirstd(keys=["image"], channel_dim=-1),
        transforms.ResizeD(keys=["image"], spatial_size=(64, 64)),
        ############################################################

        transforms.ScaleIntensityRanged(keys=["image"], a_min=0.0, a_max=255.0, b_min=0.0, b_max=1.0, clip=True),
        transforms.RandAffined(
            keys=["image"],
            rotate_range=[(-np.pi / 36, np.pi / 36), (-np.pi / 36, np.pi / 36)],
            translate_range=[(-1, 1), (-1, 1)],
            scale_range=[(-0.05, 0.05), (-0.05, 0.05)],
            spatial_size=[64, 64],
            padding_mode="zeros",
            prob=0.5,
        ),
    ]
)

############################################################
#train_ds = CacheDataset(data=train_datalist, transform=train_transforms)
#train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=4, persistent_workers=True)
train_ds = Dataset(data=train_datalist, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2, persistent_workers=True)
############################################################

In [ ]:
############################################################
#val_data = MedNISTDataset(root_dir=root_dir, section="validation", download=True, progress=False, seed=0)
#val_datalist = [{"image": item["image"]} for item in val_data.data if item["class_name"] == "Hand"]

val_data = PathMNIST(split="val", download=True, size=28)

val_datalist = [
    {"image": img}
    for img, label in zip(val_data.imgs, val_data.labels)
    if label[0] == CLASE_ELEGIDA
][:1000]
############################################################

val_transforms = transforms.Compose(
    [
        ############################################################
        #transforms.LoadImaged(keys=["image"]),
        #transforms.EnsureChannelFirstd(keys=["image"]),
        #transforms.Lambda(func=rgb_to_gray_cv2),
        transforms.EnsureChannelFirstd(keys=["image"], channel_dim=-1),
        transforms.ResizeD(keys=["image"], spatial_size=(64, 64)),
        ############################################################
        transforms.ScaleIntensityRanged(keys=["image"], a_min=0.0, a_max=255.0, b_min=0.0, b_max=1.0, clip=True),
    ]
)
############################################################
#val_ds = CacheDataset(data=val_datalist, transform=val_transforms)
#val_loader = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=4, persistent_workers=True)
val_ds = Dataset(data=val_datalist, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False, num_workers=2, persistent_workers=True)
############################################################

### Visualisation of the training images

In [ ]:
'''
check_data = first(train_loader)
print(f"batch shape: {check_data['image'].shape}")
image_visualisation = torch.cat(
    [check_data["image"][0, 0], check_data["image"][1, 0], check_data["image"][2, 0], check_data["image"][3, 0]], dim=1
)
plt.figure("training images", (12, 6))
plt.imshow(image_visualisation, vmin=0, vmax=1, cmap="gray")
plt.axis("off")
plt.tight_layout()
plt.show()
'''

check_data = first(train_loader)
img_tensor = check_data["image"]
print(f"Batch shape: {img_tensor.shape}")

images_to_show = [img_tensor[i] for i in range(4)]


if img_tensor.shape[1] == 3:
    combined = torch.cat(images_to_show, dim=2)
    image_visualisation = combined.permute(1, 2, 0).cpu().numpy()
    cmap = None
else:
    combined = torch.cat(images_to_show, dim=2)
    image_visualisation = combined.squeeze().cpu().numpy()
    cmap = "gray"

plt.figure("Visualización PathMNIST", figsize=(12, 6))
plt.imshow(image_visualisation, vmin=0, vmax=1, cmap=cmap)
plt.axis("off")
plt.title(f"Muestra de entrenamiento - Clase: {CLASE_ELEGIDA}")
plt.tight_layout()
plt.show()

### Define network, scheduler, optimizer, and inferer
At this step, we instantiate the MONAI components to create a DDPM, the UNET, the noise scheduler, and the inferer used for training and sampling. We are using
the original DDPM scheduler containing 1000 timesteps in its Markov chain, and a 2D UNET with attention mechanisms
in the 2nd and 3rd levels, each with 1 attention head.

In [ ]:
device = torch.device("cuda")

model = DiffusionModelUNet(
    spatial_dims=2,
    ############################################################
    #in_channels=1,
    #out_channels=1,
    #num_channels=(128, 256, 256),
    in_channels=3,
    out_channels=3,
    channels=(128, 256, 256),
    ############################################################
    attention_levels=(False, True, True),
    num_res_blocks=1,
    num_head_channels=256,
)
model.to(device)

scheduler = DDPMScheduler(num_train_timesteps=1000)

optimizer = torch.optim.Adam(params=model.parameters(), lr=2.5e-5)

inferer = DiffusionInferer(scheduler)

### Model training
Here, we are training our model for 75 epochs (training time: ~50 minutes).

If you would like to skip the training and use a pre-trained model instead, set `use_pretrained=True`. This model was trained using the code in `tutorials/generative/distributed_training/ddpm_training_ddp.py`

In [ ]:
use_pretrained = False

EPOCAS_ELEGIDAS = 35

if use_pretrained:
    model = torch.hub.load("marksgraham/pretrained_generative_models:v0.2", model="ddpm_2d", verbose=True).to(device)
else:
    n_epochs = EPOCAS_ELEGIDAS
    val_interval = 5
    epoch_loss_list = []
    val_epoch_loss_list = []

    scaler = GradScaler()
    total_start = time.time()
    for epoch in range(n_epochs):
        model.train()
        epoch_loss = 0
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), ncols=70)
        progress_bar.set_description(f"Epoch {epoch}")
        for step, batch in progress_bar:
            images = batch["image"].to(device)
            optimizer.zero_grad(set_to_none=True)

            with autocast(enabled=True):
                # Generate random noise
                noise = torch.randn_like(images).to(device)

                # Create timesteps
                timesteps = torch.randint(
                    0, inferer.scheduler.num_train_timesteps, (images.shape[0],), device=images.device
                ).long()

                # Get model prediction
                noise_pred = inferer(inputs=images, diffusion_model=model, noise=noise, timesteps=timesteps)

                loss = F.mse_loss(noise_pred.float(), noise.float())

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            epoch_loss += loss.item()

            progress_bar.set_postfix({"loss": epoch_loss / (step + 1)})
        epoch_loss_list.append(epoch_loss / (step + 1))

        if (epoch + 1) % val_interval == 0:
            model.eval()
            val_epoch_loss = 0
            for step, batch in enumerate(val_loader):
                images = batch["image"].to(device)
                with torch.no_grad():
                    with autocast(enabled=True):
                        noise = torch.randn_like(images).to(device)
                        timesteps = torch.randint(
                            0, inferer.scheduler.num_train_timesteps, (images.shape[0],), device=images.device
                        ).long()
                        noise_pred = inferer(inputs=images, diffusion_model=model, noise=noise, timesteps=timesteps)
                        val_loss = F.mse_loss(noise_pred.float(), noise.float())

                val_epoch_loss += val_loss.item()
                progress_bar.set_postfix({"val_loss": val_epoch_loss / (step + 1)})
            val_epoch_loss_list.append(val_epoch_loss / (step + 1))

            # Sampling image during training
            noise = torch.randn((1, 3, 64, 64))
            noise = noise.to(device)
            scheduler.set_timesteps(num_inference_steps=1000)
            with autocast(enabled=True):
                image = inferer.sample(input_noise=noise, diffusion_model=model, scheduler=scheduler)

            plt.figure(figsize=(2, 2))
            ############################################################
            #plt.imshow(image[0, 0].cpu(), vmin=0, vmax=1, cmap="gray")
            plt.imshow(image[0].permute(1, 2, 0).cpu().numpy())
            ############################################################
            plt.tight_layout()
            plt.axis("off")
            plt.show()

    total_time = time.time() - total_start
    print(f"train completed, total time: {total_time}.")

### Learning curves

In [ ]:
if not use_pretrained:
    plt.style.use("seaborn-v0_8")
    plt.title("Learning Curves", fontsize=20)
    plt.plot(np.linspace(1, n_epochs, n_epochs), epoch_loss_list, color="C0", linewidth=2.0, label="Train")
    plt.plot(
        np.linspace(val_interval, n_epochs, int(n_epochs / val_interval)),
        val_epoch_loss_list,
        color="C1",
        linewidth=2.0,
        label="Validation",
    )
    plt.yticks(fontsize=12)
    plt.xticks(fontsize=12)
    plt.xlabel("Epochs", fontsize=16)
    plt.ylabel("Loss", fontsize=16)
    plt.legend(prop={"size": 14})
    plt.show()

### Plotting sampling process along DDPM's Markov chain

In [ ]:
model.eval()
############################################################
#noise = torch.randn((1, 1, 64, 64))
noise = torch.randn((1, 3, 64, 64))
############################################################
noise = noise.to(device)
scheduler.set_timesteps(num_inference_steps=1000)
with autocast(enabled=True):
    image, intermediates = inferer.sample(
        input_noise=noise, diffusion_model=model, scheduler=scheduler, save_intermediates=True, intermediate_steps=100
    )

chain = torch.cat(intermediates, dim=-1)

plt.style.use("default")
############################################################
#plt.imshow(chain[0, 0].cpu(), vmin=0, vmax=1, cmap="gray")
plt.imshow(chain[0].permute(1, 2, 0).cpu().numpy())
############################################################
plt.tight_layout()
plt.axis("off")
plt.show()

### Cleanup data directory

Remove directory if a temporary was used.

In [ ]:
if directory is None:
    shutil.rmtree(root_dir)